In [1]:
import pyspark
from pyspark.sql import SparkSession
import traceback

# Creating a Spark session

In [2]:
spark = SparkSession.builder.appName('practice').getOrCreate()
spark

24/03/09 16:44:43 WARN Utils: Your hostname, Ds-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.29.89 instead (on interface en0)
24/03/09 16:44:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/09 16:44:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 49916)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/Users/dsaipawan/Documents/python-learing/big-data/big_data_env/lib/python3.11/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/Users/dsa

# Reading a csv file

In [3]:
df = spark.read.csv('test1.csv', header= True, inferSchema=True) 
#inferSchema option tells the reader to infer data types from the source file.

df.show()
df.printSchema()

+---+-----+---+----------+
| id|names|age|experience|
+---+-----+---+----------+
|  0|  sai| 12|         1|
|  1|pawan| 24|         2|
|  2|    d| 36|         3|
+---+-----+---+----------+

root
 |-- id: integer (nullable = true)
 |-- names: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)



In [4]:
df_name_exp = df.select(['names', 'experience'])
df_name_exp.show()

print(type(df_name_exp))
df.dtypes
df.describe().show()

+-----+----------+
|names|experience|
+-----+----------+
|  sai|         1|
|pawan|         2|
|    d|         3|
+-----+----------+

<class 'pyspark.sql.dataframe.DataFrame'>


24/03/09 16:44:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---+-----+----+----------+
|summary| id|names| age|experience|
+-------+---+-----+----+----------+
|  count|  3|    3|   3|         3|
|   mean|1.0| NULL|24.0|       2.0|
| stddev|1.0| NULL|12.0|       1.0|
|    min|  0|    d|  12|         1|
|    max|  2|  sai|  36|         3|
+-------+---+-----+----+----------+



# Columns in DataFrame

In [5]:
# Picking up a singel column will change the datatype to column not dataframe.
names = df_name_exp['names']
print(names, type(names))

try:
    # Show only works on the dataframe not on columns
    names.show()
except TypeError as e:
    print(traceback.print_exception(e))



Column<'names'> <class 'pyspark.sql.column.Column'>
None


Traceback (most recent call last):
  File "/var/folders/71/c7pdh8bj3p1chk4tv474y1g80000gn/T/ipykernel_4733/3921233689.py", line 7, in <module>
    names.show()
TypeError: 'Column' object is not callable


### Adding the columns

In [6]:
df_new_clm = df.withColumn('Experience after 2 years', df['experience'] + 2)
df_new_clm.show()

+---+-----+---+----------+------------------------+
| id|names|age|experience|Experience after 2 years|
+---+-----+---+----------+------------------------+
|  0|  sai| 12|         1|                       3|
|  1|pawan| 24|         2|                       4|
|  2|    d| 36|         3|                       5|
+---+-----+---+----------+------------------------+



### Droping the Column

In [7]:
df_new_clm.drop('Experience after 2 years').show()

+---+-----+---+----------+
| id|names|age|experience|
+---+-----+---+----------+
|  0|  sai| 12|         1|
|  1|pawan| 24|         2|
|  2|    d| 36|         3|
+---+-----+---+----------+



### Renaming the Column

In [8]:
df.withColumnRenamed('names', 'name').show()

+---+-----+---+----------+
| id| name|age|experience|
+---+-----+---+----------+
|  0|  sai| 12|         1|
|  1|pawan| 24|         2|
|  2|    d| 36|         3|
+---+-----+---+----------+



### Adding new rows using Union

In [9]:
from pyspark.sql import Row

new_rows = spark.createDataFrame(
    [
        (3, 'saipawan', 34, 4),
        (4, 'pawand', None, 5),
        (5, 'said', None, None),
        (None, None, None, None)
    ],
     df.columns
    )
new_rows.show()

df = df.union(new_rows)

df.show()


+----+--------+----+----------+
|  id|   names| age|experience|
+----+--------+----+----------+
|   3|saipawan|  34|         4|
|   4|  pawand|NULL|         5|
|   5|    said|NULL|      NULL|
|NULL|    NULL|NULL|      NULL|
+----+--------+----+----------+

+----+--------+----+----------+
|  id|   names| age|experience|
+----+--------+----+----------+
|   0|     sai|  12|         1|
|   1|   pawan|  24|         2|
|   2|       d|  36|         3|
|   3|saipawan|  34|         4|
|   4|  pawand|NULL|         5|
|   5|    said|NULL|      NULL|
|NULL|    NULL|NULL|      NULL|
+----+--------+----+----------+



In [10]:
data1 = [[1, 'sai', 2000], [2, 'pawan', 3000]]
schema1 = ['_id', 'name', 'salary']

data2 = [[3, 'dsai', 4000], [4, 'saipawan', 5000], [2, 'pawan', 3000]]
schema2 = ['_id', 'name', 'salary']

df1 = spark.createDataFrame(data1, schema1)
df2 = spark.createDataFrame(data2, schema2)

# Unlike sql, the union function does not remove the duplicate rows
df1.union(df2).show()
df1.unionAll(df2).show()

+---+--------+------+
|_id|    name|salary|
+---+--------+------+
|  1|     sai|  2000|
|  2|   pawan|  3000|
|  3|    dsai|  4000|
|  4|saipawan|  5000|
|  2|   pawan|  3000|
+---+--------+------+

+---+--------+------+
|_id|    name|salary|
+---+--------+------+
|  1|     sai|  2000|
|  2|   pawan|  3000|
|  3|    dsai|  4000|
|  4|saipawan|  5000|
|  2|   pawan|  3000|
+---+--------+------+



### Union by name

In [11]:
data1 = [[1, 'sai', 2000], [2, 'pawan', 3000]]
schema1 = ['_id', 'name', 'salary']

data2 = [[3, 'dsai', 'male'], [4, 'saipawan', 'male'], [5, 'xyz', 'female']]
schema2 = ['_id', 'name', 'gender']

df1 = spark.createDataFrame(data1, schema1)
df2 = spark.createDataFrame(data2, schema2)

try:
    # This will throw an error as there is an mismatch in the col name.
    df1.unionByName(df2)
except Exception as e:
    print(e)


# This will merge the dataFrames using the column name.
df1.unionByName(df2, allowMissingColumns= True).show()

Cannot resolve column name "salary" among (_id, name, gender).
+---+--------+------+------+
|_id|    name|salary|gender|
+---+--------+------+------+
|  1|     sai|  2000|  NULL|
|  2|   pawan|  3000|  NULL|
|  3|    dsai|  NULL|  male|
|  4|saipawan|  NULL|  male|
|  5|     xyz|  NULL|female|
+---+--------+------+------+



# Distinct and DropDuplicates

In [28]:
data = [[1, 'sai', 'male', 2000],[2, 'pawan', 'male', 3000], [2, 'pawan', 'male', 3000], [3, 'd', 'female', 4000]]
schema = ['_id', 'name', 'gender', 'salary']

df = spark.createDataFrame(data, schema)
df.show()

+---+-----+------+------+
|_id| name|gender|salary|
+---+-----+------+------+
|  1|  sai|  male|  2000|
|  2|pawan|  male|  3000|
|  2|pawan|  male|  3000|
|  3|    d|female|  4000|
+---+-----+------+------+



In [29]:
df.distinct().show()

df.dropDuplicates().show()
# now it will only show the 1st occurance
df.dropDuplicates(['gender']).show()
df.dropDuplicates(['gender', 'salary']).show()


+---+-----+------+------+
|_id| name|gender|salary|
+---+-----+------+------+
|  1|  sai|  male|  2000|
|  2|pawan|  male|  3000|
|  3|    d|female|  4000|
+---+-----+------+------+

+---+-----+------+------+
|_id| name|gender|salary|
+---+-----+------+------+
|  1|  sai|  male|  2000|
|  2|pawan|  male|  3000|
|  3|    d|female|  4000|
+---+-----+------+------+

+---+----+------+------+
|_id|name|gender|salary|
+---+----+------+------+
|  3|   d|female|  4000|
|  1| sai|  male|  2000|
+---+----+------+------+

+---+-----+------+------+
|_id| name|gender|salary|
+---+-----+------+------+
|  3|    d|female|  4000|
|  1|  sai|  male|  2000|
|  2|pawan|  male|  3000|
+---+-----+------+------+



# Manipulating NULL values

In [49]:
df = spark.createDataFrame(
    [
        (0, 'sai', 12, 1),
        (1, 'pawan', 24, 2),
        (2, 'd', 36, 3),
        (3, 'saipawan', 34, 4),
        (4, 'pawand', None, 5),
        (5, 'said', None, None),
        (None, None, None, None)
    ],
     ['id', 'name','age', 'experience']
    )
df.show()
df.printSchema()

+----+--------+----+----------+
|  id|    name| age|experience|
+----+--------+----+----------+
|   0|     sai|  12|         1|
|   1|   pawan|  24|         2|
|   2|       d|  36|         3|
|   3|saipawan|  34|         4|
|   4|  pawand|NULL|         5|
|   5|    said|NULL|      NULL|
|NULL|    NULL|NULL|      NULL|
+----+--------+----+----------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- experience: long (nullable = true)



### Droping Null Values

In [38]:
# This will drop all the rows with Null values in it.
df.na.drop().show()

+---+--------+---+----------+
| id|    name|age|experience|
+---+--------+---+----------+
|  0|     sai| 12|         1|
|  1|   pawan| 24|         2|
|  2|       d| 36|         3|
|  3|saipawan| 34|         4|
+---+--------+---+----------+



In [39]:
df.na.drop(how='all').show()

df.na.drop(how='any').show()  # this is the default values

+---+--------+----+----------+
| id|    name| age|experience|
+---+--------+----+----------+
|  0|     sai|  12|         1|
|  1|   pawan|  24|         2|
|  2|       d|  36|         3|
|  3|saipawan|  34|         4|
|  4|  pawand|NULL|         5|
|  5|    said|NULL|      NULL|
+---+--------+----+----------+

+---+--------+---+----------+
| id|    name|age|experience|
+---+--------+---+----------+
|  0|     sai| 12|         1|
|  1|   pawan| 24|         2|
|  2|       d| 36|         3|
|  3|saipawan| 34|         4|
+---+--------+---+----------+



In [40]:
df.na.drop(how='any',thresh=2).show()
# This will delete all the rows with dosent have more than <thresh> non null values.

+---+--------+----+----------+
| id|    name| age|experience|
+---+--------+----+----------+
|  0|     sai|  12|         1|
|  1|   pawan|  24|         2|
|  2|       d|  36|         3|
|  3|saipawan|  34|         4|
|  4|  pawand|NULL|         5|
|  5|    said|NULL|      NULL|
+---+--------+----+----------+



In [41]:
df.na.drop(how='any', subset=['experience']).show()
# Specify which columns you want to focus on.

+---+--------+----+----------+
| id|    name| age|experience|
+---+--------+----+----------+
|  0|     sai|  12|         1|
|  1|   pawan|  24|         2|
|  2|       d|  36|         3|
|  3|saipawan|  34|         4|
|  4|  pawand|NULL|         5|
+---+--------+----+----------+



### Filling the missing values

In [56]:
# fill and fillNa will work the same way.
df.fillna('Missing Values', subset=['name']).show()

df.na.fill( 0 , subset=['age', 'experience']).show()

+----+--------------+----+----------+
|  id|          name| age|experience|
+----+--------------+----+----------+
|   0|           sai|  12|         1|
|   1|         pawan|  24|         2|
|   2|             d|  36|         3|
|   3|      saipawan|  34|         4|
|   4|        pawand|NULL|         5|
|   5|          said|NULL|      NULL|
|NULL|Missing Values|NULL|      NULL|
+----+--------------+----+----------+

+----+--------+---+----------+
|  id|    name|age|experience|
+----+--------+---+----------+
|   0|     sai| 12|         1|
|   1|   pawan| 24|         2|
|   2|       d| 36|         3|
|   3|saipawan| 34|         4|
|   4|  pawand|  0|         5|
|   5|    said|  0|         0|
|NULL|    NULL|  0|         0|
+----+--------+---+----------+



24/03/09 18:32:10 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 276988 ms exceeds timeout 120000 ms
24/03/09 18:32:10 WARN SparkContext: Killing executors is not supported by current scheduler.
24/03/09 18:32:10 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
from pyspark.ml.feature import Imputer

cols = ['age', 'experience']

imputer = Imputer(
    inputCols = cols,
    outputCols = [ f'{col}_impute' for col in cols]
).setStrategy('mean')

imputer.fit(df).transform(df).show()

+----+--------+----+----------+----------+-----------------+
| _c0|   names| age|experience|age_impute|experience_impute|
+----+--------+----+----------+----------+-----------------+
|   0|     sai|  12|         1|        12|                1|
|   1|   pawan|  24|         2|        24|                2|
|   2|       d|  36|         3|        36|                3|
|   3|saipawan|  34|         4|        34|                4|
|   4|  pawand|NULL|         5|        26|                5|
|   5|    said|NULL|      NULL|        26|                3|
|NULL|    NULL|NULL|      NULL|        26|                3|
+----+--------+----+----------+----------+-----------------+



24/03/04 19:38:39 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , names, age, experience
 Schema: _c0, names, age, experience
Expected: _c0 but found: 
CSV file: file:///Users/dsaipawan/Documents/python-learing/big-data/test1.csv


# Filter/where on DataFrame

In [ ]:
df.show()

# Both filter and where works the same.
df.filter(
    (df['age'] > 15) & (df['age'] < 35)
).show()
# other operations are &,|, ==, ~

df.filter(
    ~(df['age'] < 15)
).show()

+---+-----+---+----------+
| id|names|age|experience|
+---+-----+---+----------+
|  0|  sai| 12|         1|
|  1|pawan| 24|         2|
|  2|    d| 36|         3|
+---+-----+---+----------+

+---+-----+---+----------+
| id|names|age|experience|
+---+-----+---+----------+
|  1|pawan| 24|         2|
+---+-----+---+----------+

+---+-----+---+----------+
| id|names|age|experience|
+---+-----+---+----------+
|  1|pawan| 24|         2|
|  2|    d| 36|         3|
+---+-----+---+----------+



24/03/09 04:12:27 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 607222 ms exceeds timeout 120000 ms
24/03/09 04:12:27 WARN SparkContext: Killing executors is not supported by current scheduler.
24/03/09 04:12:34 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

# GroupBy and Aggrigate Functions

In [ ]:
data = [['Sai', 'Maths', 10000],
['Sai', 'English', 5000],
['Pawan', 'Science', 4000],
['Sai', 'Science', 4000],
['pawansai', 'Maths', 3000],
['saipawan', 'Maths', 20000],
['saipawan', 'English', 10000],
['saipawan', 'Science', 5000],
['dsai', 'Maths', 10000],
['dsai', 'Science', 2000]]

df = spark.createDataFrame(data, ['Name', 'Department', 'Salary'],)
df.show()


+--------+----------+------+
|    Name|Department|Salary|
+--------+----------+------+
|     Sai|     Maths| 10000|
|     Sai|   English|  5000|
|   Pawan|   Science|  4000|
|     Sai|   Science|  4000|
|pawansai|     Maths|  3000|
|saipawan|     Maths| 20000|
|saipawan|   English| 10000|
|saipawan|   Science|  5000|
|    dsai|     Maths| 10000|
|    dsai|   Science|  2000|
+--------+----------+------+



In [ ]:
df.groupBy('Name').sum().show()

df.groupBy('Department').avg().sort('avg(Salary)').show()

+--------+-----------+
|    Name|sum(Salary)|
+--------+-----------+
|     Sai|      19000|
|   Pawan|       4000|
|pawansai|       3000|
|saipawan|      35000|
|    dsai|      12000|
+--------+-----------+

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|   Science|     3750.0|
|   English|     7500.0|
|     Maths|    10750.0|
+----------+-----------+



In [ ]:
from pyspark.sql.functions import count, min, max

# Agg function are used to appply multipe aggrigate operation on one grp by action.
# * means all the columns.
df.groupby('Department').agg(count('*').alias("Count of all employes"), min('salary'), max('salary')).show()

+----------+---------------------+-----------+-----------+
|Department|Count of all employes|min(salary)|max(salary)|
+----------+---------------------+-----------+-----------+
|     Maths|                    4|       3000|      20000|
|   English|                    2|       5000|      10000|
|   Science|                    4|       2000|       5000|
+----------+---------------------+-----------+-----------+



# Pivot Function

In [20]:
data = [[1,'Sai', 'Maths', 'male'],
[2, 'Sai', 'English', 'male'],
[3,'Pawan', 'Science', 'female'],
[4, 'Sai', 'Science', 'female'],
[5, 'pawansai', 'Maths', 'female'],
[6, 'saipawan', 'Maths', 'male'],
[7, 'saipawan', 'English', 'male'],
[8, 'saipawan', 'Science', 'male'],
[9, 'dsai', 'Maths', 'female'],
[10, 'dsai', 'Science', 'female']]

df = spark.createDataFrame(data, ['id', 'Name', 'Department', 'Gender'],)
df.show()

+---+--------+----------+------+
| id|    Name|Department|Gender|
+---+--------+----------+------+
|  1|     Sai|     Maths|  male|
|  2|     Sai|   English|  male|
|  3|   Pawan|   Science|female|
|  4|     Sai|   Science|female|
|  5|pawansai|     Maths|female|
|  6|saipawan|     Maths|  male|
|  7|saipawan|   English|  male|
|  8|saipawan|   Science|  male|
|  9|    dsai|     Maths|female|
| 10|    dsai|   Science|female|
+---+--------+----------+------+



In [21]:
df.groupBy(['Department', 'Gender']).count().show()

# Pivot the gender column to get number of males and females in one department
pivot_df = df.groupBy('Department').pivot('Gender').count()
pivot_df.show()

# If I have to dsiaply only number of males in the department.
df.groupBy('Department').pivot('Gender', ['male']).count().show()


+----------+------+-----+
|Department|Gender|count|
+----------+------+-----+
|     Maths|  male|    2|
|   English|  male|    2|
|   Science|female|    3|
|     Maths|female|    2|
|   Science|  male|    1|
+----------+------+-----+

+----------+------+----+
|Department|female|male|
+----------+------+----+
|   Science|     3|   1|
|   English|  NULL|   2|
|     Maths|     2|   2|
+----------+------+----+

+----------+----+
|Department|male|
+----------+----+
|   Science|   1|
|   English|   2|
|     Maths|   2|
+----------+----+



In [27]:
# Unpivot functions
# There is not unpivot functionality in pyspark, we have to use stack functionality to unpivot.

pivot_df.show()

# Now I want this above dataframe with columns as Department, Gneder, Count as below.
"""
|Department|Gender|count|
+----------+------+-----+
|     Maths|  male|    2|
|   English|  male|    2|
|   Science|female|    3|
|     Maths|female|    2|
|   Science|  male|    1|
"""
from pyspark.sql.functions import expr, stack

# expr fucntion takes a sting as an input but treat that as a python code.
pivot_df.select('Department', expr("stack(2, 'M', male, 'F', female) as (gender, count)")).show()
# here 2 represents number of columns values that we are unpivoting, followed by their names.

+----------+------+----+
|Department|female|male|
+----------+------+----+
|   Science|     3|   1|
|   English|  NULL|   2|
|     Maths|     2|   2|
+----------+------+----+

+----------+------+-----+
|Department|gender|count|
+----------+------+-----+
|   Science|     M|    1|
|   Science|     F|    3|
|   English|     M|    2|
|   English|     F| NULL|
|     Maths|     M|    2|
|     Maths|     F|    2|
+----------+------+-----+



# OrderBy and Sort functions

In [ ]:
data = [[1, 'sai', 'male', 5000],[2, 'pawan', 'male', 2000], [2, 'saipawan', 'male', 10000]]
schema = ['_id', 'name', 'gender', 'salary']

df = spark.createDataFrame(data, schema)
df.show()

+---+--------+------+------+
|_id|    name|gender|salary|
+---+--------+------+------+
|  1|     sai|  male|  5000|
|  2|   pawan|  male|  2000|
|  2|saipawan|  male| 10000|
+---+--------+------+------+



In [ ]:
df.sort(df.salary).show()
df.sort(df.salary.desc()).show()

+---+--------+------+------+
|_id|    name|gender|salary|
+---+--------+------+------+
|  2|   pawan|  male|  2000|
|  1|     sai|  male|  5000|
|  2|saipawan|  male| 10000|
+---+--------+------+------+

+---+--------+------+------+
|_id|    name|gender|salary|
+---+--------+------+------+
|  2|saipawan|  male| 10000|
|  1|     sai|  male|  5000|
|  2|   pawan|  male|  2000|
+---+--------+------+------+



In [ ]:
df.orderBy(df.salary).show()
df.orderBy(df.salary.desc()).show()

+---+--------+------+------+
|_id|    name|gender|salary|
+---+--------+------+------+
|  2|   pawan|  male|  2000|
|  1|     sai|  male|  5000|
|  2|saipawan|  male| 10000|
+---+--------+------+------+

+---+--------+------+------+
|_id|    name|gender|salary|
+---+--------+------+------+
|  2|saipawan|  male| 10000|
|  1|     sai|  male|  5000|
|  2|   pawan|  male|  2000|
+---+--------+------+------+

